Importing the dependencies

In [3]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saurabh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [13]:
# loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('train.csv')


In [11]:
news_dataset.shape

(20800, 5)

In [14]:
# print first 5 rows of dataframe
news_dataset.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [15]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [16]:
# replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [17]:
# merging the author name , news title and text
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']+' '+['text']

In [18]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799         David Swanson What Keeps the F-35 Alive text
Name: content, Length: 20800, dtype: object


In [19]:
# separating the data & label
X = news_dataset.drop(columns='label',axis = 1)
Y = news_dataset['label']

In [20]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
stemming is the process of reducing a word to its Root word

In [21]:
port_stem = PorterStemmer()

In [22]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [23]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [24]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2              consortiumnew com truth might get fire text
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                       david swanson keep f aliv text
Name: content, Length: 20800, dtype: object


In [25]:
# separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values



In [26]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet text'
 'daniel j flynn flynn hillari clinton big woman campu breitbart text'
 'consortiumnew com truth might get fire text' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time text'
 'alex ansari nato russia hold parallel exercis balkan text'
 'david swanson keep f aliv text']


In [27]:
print(Y)

[1 0 1 ... 0 1 1]


In [28]:
Y.shape

(20800,)

In [29]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [30]:
print(X)

  (0, 15686)	0.28459310610746597
  (0, 15157)	0.042512997608153684
  (0, 13473)	0.2563576887639814
  (0, 8909)	0.36326765817388257
  (0, 8630)	0.29186103457104173
  (0, 7692)	0.24762811546514382
  (0, 7005)	0.2185439295563331
  (0, 4973)	0.23310602826664156
  (0, 3792)	0.27028866271677826
  (0, 3600)	0.3595685437120402
  (0, 2959)	0.24662184369143414
  (0, 2483)	0.36731957961993644
  (0, 267)	0.2698570549730114
  (1, 16799)	0.30034150642856594
  (1, 15157)	0.049987914351755466
  (1, 6816)	0.19022790354246966
  (1, 5503)	0.7134368965283879
  (1, 3568)	0.26340796919968285
  (1, 2813)	0.19070702460034808
  (1, 2223)	0.3822535557936904
  (1, 1894)	0.1550256900710693
  (1, 1497)	0.2936216176510346
  (2, 15611)	0.41451165334767964
  (2, 15157)	0.06715924123282074
  (2, 9620)	0.49240070572069106
  :	:
  (20797, 12138)	0.24760537951536501
  (20797, 10306)	0.08032330697529784
  (20797, 9588)	0.17442865113864528
  (20797, 9518)	0.29520913518971514
  (20797, 8988)	0.3613500906373095
  (20797, 836

Splitting the dataset to training & test data

In [32]:
X_train, X_test, Y_train , Y_test = train_test_split(X,Y,test_size = 0.2,stratify=Y,random_state = 2)

Training the model: Logistic Regression Model

In [33]:
model = LogisticRegression()

In [34]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation

In [35]:
# accuracy score on the training data
X_train_prediction =  model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [36]:
print('Accuracy score of training data:',training_data_accuracy)

Accuracy score of training data: 0.9878605769230769


In [37]:
# accuracy score on the test data
X_test_prediction =  model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [38]:
print('Accuracy score of test data:',test_data_accuracy)

Accuracy score of test data: 0.9798076923076923


Making a predictive system:
Real : 0
Fake: 1

In [43]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0 ):
    print('The news is Real')
else:
    print('The news is Fake')

[0]
The news is Real


In [42]:
print(Y_test[1])

0
